In [1]:
from pyomo.environ import * 
from pyomo.dae import *
from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp


In [8]:
model = ConcreteModel()
model.time = ContinuousSet(bounds=(0,10))
model.X = Var(model.time)
model.a = Var(within=Reals)
model.scale = Param(initialize=1, mutable=True)

def _intX(m,t):
    return m.X[t]
model.intX = Integral(model.time, wrt=model.time, rule=_intX)

def _con1(m):
    return m.a == m.intX + m.scale 
model.con = Constraint(rule=_con1)

def _obj(m):
    return m.a
#model.obj = Objective(rule=_obj)

model.Obj = Objective(expr=0, sense=minimize)

In [9]:
discretizer = TransformationFactory('dae.collocation')
discretizer.apply_to(model,nfe=20,wrt=model.time,scheme='LAGRANGE-RADAU', ncp=3)


In [10]:
solver = SolverFactory("ipopt")
solver.solve(model, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1, 'Number of variables': 62, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.052832603454589844}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
print(value(model.intX))

-0.6337995648274556


In [12]:
### run get_dsdp
param = {"scale": 1}
param_list = ["scale"]

dsdp, col = get_dsdp(model, param_list, param, tee=True)

W[K_AUG]...	[K_AUG_ASL]No n_rhs declared.
W[K_AUG]...	[K_AUG_ASL]Target log10mu:= -11.
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zL_in suffix declared, setting zL = 0.
W[K_AUG_ASL]...	[K_AUG_ASL]No ipopt_zU_in suffix declared, setting zU = 0.
W[K_AUG]...	[K_AUG_ASL]No f_timestamp suffix declared, Fallback to default writing mode.
I[K_AUG]...	[K_AUG_ASL] Filename for dot_sens dsdp_in_.in
W[K_AUG]...	[K_AUG_ASL]dsdp for linear C(x) + I*p = 0 override.
W[K_AUG]...	[INERTIA_STRATEGY]Wrong inertia(neig < m).
W[K_AUG]...	[INERTIA_STRATEGY]Attempting to make i_pert->d_c > 0.
W[K_AUG]...	[MA57_DRIVER]

		Could not fix the accuracy of the problem.
		Try re-writing the problem or give a different point or change "max_refinement_steps"
		Current residual ratio 0.414214; Max residual ratio 1e-10.

[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/
: dsdp_mode=
dsdp_mode=
I[K_AUG]...	[K_AUG_ASL]File read successfully.
I[K_AUG]...	[K_AUG_ASL]Number of Right hand sides: 0
I[K_AU